# Road Following 

If you've run through the collision avoidance sample, your should be familiar following three steps

(訳) collision avoidanceを実行していれば、今回のこの3ステップには馴染みがあるでしょう。

1.  Data collection
2.  Training
3.  Deployment

(訳)
1. Data収集
2. 学習
3. デプロイ

In this notebook, we'll do the same exact thing!  Except, instead of classification, you'll learn a different fundamental technique, **regression**, that we'll use to
enable JetBot to follow a road (or really, any path or target point).  

(訳) このnotebookは、同じことをします。ただし、今回はJetBotが道路（または実際に任意のパスまたはターゲットポイント）をたどることができるようにするために、画像分類の代わりに、**回帰**という別の基本的な手法で学習します。

1. Place the JetBot in different positions on a path (offset from center, different angles, etc)

(訳) 1. JetBotをパス上のさまざまな位置に配置します（中心からのオフセット、さまざまな角度など）

>  Remember from collision avoidance, data variation is key!

(訳) データのバリデーションこそが重要であると、collision avoidanceのときに覚えたはずです。

2. Display the live camera feed from the robot
3. Using a gamepad controller, place a 'green dot', which corresponds to the target direction we want the robot to travel, on the image.
4. Store the X, Y values of this green dot along with the image from the robot's camera

(訳)
2. robotからのライブカメラフィードを画面に表示します
3. ゲームパッドを使い(改良して本サンプルはマウスのみ)、`greenの点`を画像上でロボットが移動するターゲットの方向に移動します。
4. この`greenの点`のX、Y値をロボットのカメラからの画像と、一緒に保存します

Then, in the training notebook, we'll train a neural network to predict the X, Y values of our label.  In the live demo, we'll use
the predicted X, Y values to compute an approximate steering value (it's not 'exactly' an angle, as
that would require image calibration, but it's roughly proportional to the angle so our controller will work fine).

(訳)
notebookの学習で、ラベルのX, Yの値の予測をおこなうためにneural networkで学習します。このライブデモでは、概算のステアリング値を計算するために予測されたX、Y値を使用します。(角度として「正確に」ではありません。画像のキャリブレーションが必要になりますが、角度にほぼ比例するため、制御は正常に機能します)

So how do you decide exactly where to place the target for this example?  Here is a guide we think may help

(訳)
それでは、この例の目標点は、正確にどこに配置するのがいいでしょうか？ ここに役立つと思われるガイドがあります。

1.  Look at the live video feed from the camera
2.  Imagine the path that the robot should follow (try to approximate the distance it needs to avoid running off road etc.)
3.  Place the target as far along this path as it can go so that the robot could head straight to the target without 'running off' the road.

(訳)
1. カメラからのライブデモフィードを見ます
2. ロボットがたどるべき経路を想像してください（道路からの脱出などを回避するために必要な距離を概算してみてください）
3. ロボットが道路を「脱線」する事なく目標点にまっすぐ進むことができるように、目標点はこの経路に沿ってできるだけ遠くに配置します。

> For example, if we're on a very straight road, we could place it at the horizon.  If we're on a sharp turn, it may need to be placed closer to the robot so it doesn't run out of boundaries.

(訳)
> 例として、もし真っ直ぐのロードがあったとします、目標点は地平線にします。もし、鋭く曲がったカーブなら、脱線しないレベルで、ロボットの近くに配置する必要があるでしょう。

Assuming our deep learning model works as intended, these labeling guidelines should ensure the following:

(訳) 
ディープラーニングのモデルが意図したとおりに機能すると仮定すると、これらのラベリングガイドラインでは次のことが保証されます。

1.  The robot can safely travel directly towards the target (without going out of bounds etc.)
2.  The target will continuously progress along our imagined path

(訳)
1.ロボットは、ターゲットに向かって安全に直接移動できます（範囲外に出ることなく）。
2.目標点は、想像した経路に沿って継続的に処理されます。

What we get, is a 'carrot on a stick' that moves along our desired trajectory.  Deep learning decides where to place the carrot, and JetBot just follows it :)

(訳)
この処理で得られるものは、望む軌道に沿って動く「スティック上のニンジン」です。ディープラーニングがニンジンを配置する場所を決定し、JetBotがそれに追従します。

### Labeling example video(ラベリングのデモ動画)

Execute the block of code to see an example of how to we labeled the images.  This model worked after only 123 images :)

(訳) 
下記のコードを実行して、画像にラベルを付ける方法の例を確認します。このモデルはたった123枚の画像で動作しました:)

In [1]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FW4En6LejhI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


### Import Libraries(ライブラリのimport)

So lets get started by importing all the required libraries for "data collection" purpose. We will mainly use OpenCV to visualize and save image with labels. Libraries such as uuid, datetime are used for image naming. 

(訳)それでは、"data collection"のために必要なライブラリ全部をimportして、作業を開始します。ラベルと紐付いた画像を表示し、保存するためにOpenCVを使用します。uuid、datetimeなどのライブラリは、イメージのファイル名の命名に使用されます。

In [2]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### Display Live Camera Feed(ライブカメラフィードの表示)

First, let's initialize and display our camera like we did in the teleoperation notebook. 

(訳)　最初に、teleperation notebookで実行したようにカメラの初期化と表示をおこないます。

We use Camera Class from JetBot to enable CSI MIPI camera. Our neural network takes a 224x224 pixel image as input. We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task). In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

(訳) JetBotのカメラクラスは、CSI MIPI cameraを有効にするために使います。neural networkでは、224x224ピクセスの画像データをインプットとして使います。データセットのファイルサイズを最小化するために、カメラをそのサイズに設定します(このタスクのために動作する動作する事は確認しています) いくつかのシナリオでは、画像サイズを大きくし収集し、後で目的のサイズに縮小する方がいいかもしれません。

In [3]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.0, description='y', max=1.0, min=-1.0, step=0.001)

### Collect data(データ収集)

The following block of code will display the live image feed, as well as the number of images we've saved.  We store
the target X, Y values by

(訳) 次のコードブロックは、ライブ画像フィードと保存した画像の数を表示します。ターゲットとなるX、Y値の値を下記のように保存します。

1. Place the green dot on the target
2. Press 'down' on the DPAD to save

(訳)
1. 目標点に緑のドットを移動します。
2. 保存ボタンを押します(本サンプル用に、オフィシャルのサンプルを修正しています。)

This will store a file in the ``dataset_xy`` folder with files named

``xy_<x value>_<y value>_<uuid>.jpg``

(訳)
これで、``dataset_xy``フォルダーに、

``xy_<x value>_<y value>_<uuid>.jpg``

の形式で保存されます。

When we train, we load the images and parse the x, y values from the filename

(訳)
学習時に、ファイル名からx,yの値をパースし、イメージとともに読み込みます。

In [4]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

#for b in controller.buttons:
#    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot():
    #if change['new']:
    uuid = xy_uuid(x_slider.value, y_slider.value)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

#controller.buttons[13].observe(save_snapshot, names='value')
button_layout = widgets.Layout(width='128px', height='64px')
add_button = widgets.Button(description='add', button_style='success', layout=button_layout)

display(widgets.VBox([
    target_widget,
    x_slider,
    y_slider,
    count_widget,
    add_button
]))

add_button.on_click(lambda x: save_snapshot())

#display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

### Next(次)